In [1]:
import numpy as np
# Подключаем модель нейронных сетей
from tensorflow.keras.models import Sequential
# Подключаем слои
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras import utils
# Подключаем набор данных
from tensorflow.keras.datasets import fashion_mnist
# Подключаем модель сети согласно варианту
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

In [2]:
# Загружаем тренировочные и тестовые данные
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [3]:
# Преобразуем изображения в 3 канала
x_train = np.dstack([x_train]*3)
x_test = np.dstack([x_test]*3)
# Измененяем формы изображений согласно тензорному формату
x_train = x_train.reshape(-1,28,28,3)
x_test = x_test.reshape(-1,28,28,3)
# Измененяем размер изображений (48x48) в соответствии с требованиями модели сети
x_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48, 48))) for im in x_train])
x_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48, 48))) for im in x_test])

In [4]:
# Нормализуем данные
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [5]:
# Преобразуем метки в категории
y_train = utils.to_categorical(y_train,10)
y_test = utils.to_categorical(y_test,10)

In [6]:
# Создаем экземпляр модели сети
mobileNet = ResNet50V2(weights='imagenet', include_top=False, input_shape=(48,48,3))
# Без обучения сверточной части сети
mobileNet.trainable = False

# Добавляем слои
model = Sequential()
# Добавление сверточного слоя
model.add(mobileNet)
# Преобразование двумерного массива в одномерный
model.add(Flatten())
# Добавление полносвязного слоя
# На вход 700 нейронов
model.add(Dense(700, activation='relu'))
# Слой регуляризации
model.add(Dropout(0.5))
# На выход 10 нейронов
model.add(Dense(10, activation='softmax'))

94674944/94668760 [==============================] - 1s 0us/step


In [7]:
# Компилируем модель
model.compile(loss='categorical_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [8]:
# Выводим модели
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2, 2, 2048)        23564800  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 700)               5735100   
_________________________________________________________________
dropout (Dropout)            (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                7010      
Total params: 29,306,910
Trainable params: 5,742,110
Non-trainable params: 23,564,800
_________________________________________________________________
None


In [9]:
# Обучаем нейронную сеть
model.fit(x_train, y_train, 
          batch_size=250, 
          epochs=12, 
          validation_split=0.1,
          shuffle=True)

Epoch 1/12
216/216 [==============================] - 400s 2s/step - loss: 1.2736 - accuracy: 0.6886 - val_loss: 0.5690 - val_accuracy: 0.7950
Epoch 2/12
216/216 [==============================] - 396s 2s/step - loss: 0.6277 - accuracy: 0.7764 - val_loss: 0.5120 - val_accuracy: 0.8152
Epoch 3/12
216/216 [==============================] - 395s 2s/step - loss: 0.5670 - accuracy: 0.7954 - val_loss: 0.4931 - val_accuracy: 0.8197
Epoch 4/12
216/216 [==============================] - 396s 2s/step - loss: 0.5301 - accuracy: 0.8087 - val_loss: 0.4634 - val_accuracy: 0.8278
Epoch 5/12
216/216 [==============================] - 396s 2s/step - loss: 0.5054 - accuracy: 0.8172 - val_loss: 0.4648 - val_accuracy: 0.8277
Epoch 6/12
216/216 [==============================] - 396s 2s/step - loss: 0.4893 - accuracy: 0.8213 - val_loss: 0.4455 - val_accuracy: 0.8342
Epoch 7/12
216/216 [==============================] - 396s 2s/step - loss: 0.4746 - accuracy: 0.8261 - val_loss: 0.4549 - val_accuracy: 0.8280

In [10]:
# Проверяем точность модели на тестовой выборке
scores = model.evaluate(x_test, y_test, verbose=0)
print('Доля верных ответов на тестовых данных, в процентах:', round(scores[1]*100, 4))

Доля верных ответов на тестовых данных, в процентах: 84.11
